# Weighted Least Squares

In [ ]:
from gnss_lib_py.parsers.android import AndroidDerived2021
from gnss_lib_py.algorithms.snapshot import solve_wls
from gnss_lib_py.utils.visualizations import plot_map

In [ ]:
# load Android Google Challenge data
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/android_2021/Pixel4XL_derived.csv --quiet -O "Pixel4XL_derived.csv"
derived_data = AndroidDerived2021("Pixel4XL_derived.csv", remove_timing_outliers=False)

Solve for the Weighted Least Squares position estimate simply by passing the measurement data.

In [ ]:
state = solve_wls(derived_data)

Plot the ECEF x and ECEF y computed position estimate of the receiver

In [ ]:
plot_map(state)

# Residuals

In [ ]:
from gnss_lib_py.algorithms.residuals import solve_residuals
from gnss_lib_py.utils.visualizations import plot_metric_by_constellation
galileo_data = derived_data.where("gnss_id","galileo")

Solve for residuals using the estimated state. A new "residuals_m" row is added to derived_data

In [ ]:
solve_residuals(galileo_data, state, inplace=True)

Plot the residuals using the `plot_residuals()` function. The residuals are broken up constellation and signal type and plotted on separate figures.

In [ ]:
figs = plot_metric_by_constellation(galileo_data, "gps_millis", "residuals_m")

# Multi-GNSS

Solve for precise ephemerides and update derived_data class using array of sp3 and clk classes.

In [ ]:
from gnss_lib_py.parsers.precise_ephemerides import parse_sp3, parse_clockfile
from gnss_lib_py.algorithms.multi_gnss import compute_sv_gnss_from_precise_eph

In [ ]:
# load Android Google Challenge data
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/ramya/multiGNSS_sp3/data/unit_test/android_2021/Pixel4_derived_clkdiscnt.csv --quiet -O "Pixel4_derived_clkdiscnt.csv"
derived_data = AndroidDerived2021("Pixel4_derived_clkdiscnt.csv", remove_timing_outliers=False)
derived_data_gps = derived_data.where("gnss_id", "gps")

In [ ]:
# download .sp3 data file
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/ramya/multiGNSS_sp3/data/unit_test/precise_ephemeris/grg21553_short.sp3 --quiet -O "grg21553_short.sp3"

# Parse precise ephemerides, i.e., satellite position, from .sp3 data
sp3data_gps = parse_sp3("grg21553_short.sp3", constellation = 'gps')

In [ ]:
# download .clk data file
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/ramya/multiGNSS_sp3/data/unit_test/precise_ephemeris/grg21553_short.clk --quiet -O "grg21553_short.clk"

# Parse precise ephemerides, i.e., satellite clock bias, from .clk data
clkdata_gps = parse_clockfile("grg21553_short.clk", constellation = 'gps')

In [ ]:
# Update derived_data_gps class with satellite information computed via precise ephemerides
derived_multi_gnss = compute_sv_gnss_from_precise_eph(derived_data_gps, sp3data_gps, \
                                                      clkdata_gps, verbose = False)